In [1]:
import torch
from transformer_lens import HookedTransformer

/opt/miniconda3/envs/tite/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model = HookedTransformer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
positive_prompt = "Love represents kindness, affection, and deep connection."
negative_prompt = "Hate represents anger, hostility, and deep resentment."

In [5]:
target_layer = 6
hook_name = f"blocks.{target_layer}.hook_resid_pre"

In [6]:
_, pos_cache = model.run_with_cache(positive_prompt)
_, neg_cache = model.run_with_cache(negative_prompt)

In [7]:
pos_act = pos_cache[hook_name].mean(dim=1) # Durchschnitt über alle Token
neg_act = neg_cache[hook_name].mean(dim=1)
steering_vector = pos_act - neg_act

In [ ]:
def steering_hook(resid_pre, hook):
    coeff = 5.0
    return resid_pre + (steering_vector * coeff)

In [12]:
test_prompt = "A cockatoo is"

output_base = model.generate(test_prompt, max_new_tokens=20, verbose=False)
print(output_base)

A cockatoo is a danger generally known ringing the bell teeming with lewd murder and committing the crime of "intentional


In [13]:
with model.hooks(fwd_hooks=[(hook_name, steering_hook)]):
    output_steered = model.generate(test_prompt, max_new_tokens=20, verbose=False)
    print(output_steered)

A cockatoo is nurturing thisforms inviting the hearts mar min foods...second perfect mom loving illusion," also is most love
